In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
# from matplotlib import cm

In [8]:
df=pd.read_csv('results_corr.csv')
df=df.drop(df.columns[0], axis=1)
df.head()

,element_count,column_count,array_time,pandas_df_time,pandas_df_compute,spark_df_time,spark_df_compute
0,10,1,0.000023,0.000052,0.000291,0.015082,0.334358
1,100,1,0.000015,0.000081,0.000149,0.019590,0.332088
2,100,10,0.000016,0.000079,0.000276,0.021800,0.315602
3,1000,1,0.000029,0.000084,0.000278,0.048502,0.314306
4,1000,10,0.000029,0.000078,0.000313,0.022881,0.305417


In [9]:
df['row_count']=df['element_count']/df['column_count']
df.sort_values(by=['row_count', 'column_count'])

,element_count,column_count,array_time,pandas_df_time,pandas_df_compute,spark_df_time,spark_df_compute,row_count
0,10,1,0.000023,0.000052,0.000291,0.015082,0.334358,10.0
2,100,10,0.000016,0.000079,0.000276,0.021800,0.315602,10.0
5,1000,100,0.000032,0.000089,0.000429,0.038831,0.364151,10.0
9,10000,1000,0.000146,0.000081,0.026096,0.211609,1.413452,10.0
1,100,1,0.000015,0.000081,0.000149,0.019590,0.332088,100.0
4,1000,10,0.000029,0.000078,0.000313,0.022881,0.305417,100.0
8,10000,100,0.000146,0.000079,0.002338,0.080792,0.342626,100.0
13,100000,1000,0.001073,0.000052,0.208883,0.697488,1.458024,100.0
3,1000,1,0.000029,0.000084,0.000278,0.048502,0.314306,1000.0
7,10000,10,0.000110,0.000052,0.000420,0.077864,0.316487,1000.0


### 2D Stacked Line Charts

In [257]:
fig, ax=plt.subplots(1, 2, sharey=True)
for each_column_ct in df['column_count'].unique(): 
    sub_df=df[df['column_count']==each_column_ct]
    ax[0].plot(np.log(sub_df['row_count']), 
               np.log(sub_df['pandas_df_compute']), label=each_column_ct)
for each_column_ct in df['column_count'].unique(): 
    sub_df=df[df['column_count']==each_column_ct]
    ax[1].plot(np.log(sub_df['row_count']), 
             np.log(sub_df['spark_df_compute']), label=each_column_ct)
plt.legend()

### 3D Scatter Plots

In [245]:
# %matplotlib notebook

fig=plt.figure()
ax=plt.axes(projection='3d')
ax.scatter3D(np.log(df['row_count']),
             np.log(df['column_count']), 
             np.log(df['pandas_df_compute']), 
             c='tab:red')
ax.scatter3D(np.log(df['row_count']),
             np.log(df['column_count']), 
             np.log(df['spark_df_compute']), 
             c='tab:blue')
# ax.scatter3D(df['row_count'],
#              df['column_count'], 
#              df['pandas_df_compute'])
ax.set_xlabel('row_count')
ax.set_ylabel('column_count')
ax.set_zlabel('df_compute')
# plt.show()

Text(0.5, 0, 'df_compute')

### 3D Surface Plots

#### Model to Draw Plane - Pandas

In [16]:
pandas_poly=PolynomialFeatures(degree=2)
pandas_x=np.log(df[['row_count', 'column_count']])
pandas_compute=np.log(df['pandas_df_compute'])
pandas_poly.fit(pandas_x)
transformed_pandas_x=pandas_poly.transform(pandas_x)

pandas_lr_poly=LinearRegression()
pandas_lr_poly.fit(transformed_pandas_x, pandas_compute)
display(pandas_lr_poly.score(transformed_pandas_x, pandas_compute))

# lr=LinearRegression()
# lr.fit(x, y)
# display(lr.score(x, y))
# display(lr.coef_)

pandas_results_poly=pandas_lr_poly.predict(transformed_pandas_x)
# results_lr=lr.predict(x)
pandas_results_df=pd.DataFrame({'y': pandas_compute, 
#                          'results_lr': results_lr, 
#                          'diff_lr': results-y, 
                         'results_poly': pandas_results_poly, 
                         'diff_poly': pandas_results_poly-pandas_compute})
# results_df

0.9912256058703817

#### Meshgrid for Plane - Pandas

In [17]:
# x_1=np.log(df['row_count'])
all_row_counts=df['row_count'].unique()
all_column_counts=df['column_count'].unique()
planes_count=10
# x_1=np.arange(4, 16, 2)
x_1=np.linspace(np.log(all_row_counts.min()), np.log(all_row_counts.max()), planes_count)
# x_2=np.log(df['column_count'])
# x_2=np.arange(0, 2.5, .5)
x_2=np.linspace(np.log(all_column_counts.min()), np.log(all_column_counts.max()), planes_count)
X_1, X_2=np.meshgrid(x_1, x_2)
r1, r2=X_1.flatten(), X_2.flatten()
r1, r2=r1.reshape(-1, 1), r2.reshape(-1, 1)
grid=np.hstack([r1, r2])

# z=lr.predict(grid)

transformed_grid=pandas_poly.transform(grid)
pandas_z=pandas_lr_poly.predict(transformed_grid)
pandas_Z=pandas_z.reshape(-1, planes_count)

#### Model to Draw Plane - Spark

In [18]:
spark_poly=PolynomialFeatures(degree=2)
spark_x=np.log(df[['row_count', 'column_count']])
spark_compute=np.log(df['spark_df_compute'])
spark_poly.fit(spark_x)
transformed_spark_x=spark_poly.transform(spark_x)

spark_lr_poly=LinearRegression()
spark_lr_poly.fit(transformed_spark_x, spark_compute)
display(spark_lr_poly.score(transformed_spark_x, spark_compute))

# lr=LinearRegression()
# lr.fit(x, y)
# display(lr.score(x, y))
# display(lr.coef_)

spark_results_poly=spark_lr_poly.predict(transformed_spark_x)
# results_lr=lr.predict(x)
spark_results_df=pd.DataFrame({'y': spark_compute, 
#                          'results_lr': results_lr, 
#                          'diff_lr': results-y, 
                         'results_poly': spark_results_poly, 
                         'diff_poly': spark_results_poly-spark_compute})
# results_df

0.9615831696797483

#### Meshgrid for Plane - Spark

In [19]:
# x_1=np.log(df['row_count'])
all_row_counts=df['row_count'].unique()
all_column_counts=df['column_count'].unique()
planes_count=10
# x_1=np.arange(4, 16, 2)
x_1=np.linspace(np.log(all_row_counts.min()), np.log(all_row_counts.max()), planes_count)
# x_2=np.log(df['column_count'])
# x_2=np.arange(0, 2.5, .5)
x_2=np.linspace(np.log(all_column_counts.min()), np.log(all_column_counts.max()), planes_count)
X_1, X_2=np.meshgrid(x_1, x_2)
r1, r2=X_1.flatten(), X_2.flatten()
r1, r2=r1.reshape(-1, 1), r2.reshape(-1, 1)
grid=np.hstack([r1, r2])

# z=lr.predict(grid)

transformed_grid=spark_poly.transform(grid)
spark_z=spark_lr_poly.predict(transformed_grid)
spark_Z=spark_z.reshape(-1, planes_count)

#### Plot

In [21]:
%matplotlib

fig=plt.figure(figsize=(7.5, 5))
#====first plot====
# ax=fig.add_subplot(1, 2, 1, projection='3d')
ax=plt.axes(projection='3d')
ax.plot_wireframe(X_1,
                X_2, 
                spark_Z, 
                color='tab:blue', 
                label='Spark DataFrame')

# ax=fig.add_subplot(1, 2, 1, projection='3d')
ax.plot_wireframe(X_1,
                X_2, 
                pandas_Z, 
                color='tab:red', 
                label='Pandas DataFrame')

xlabels_exp=sorted(df['row_count'].unique())
xlabels_exp_red=[label for idx, label in enumerate(xlabels_exp) if idx%2==0]
xlabels=np.log(xlabels_exp_red)
ax.set_xlabel('row_count')
ax.set_xticks(xlabels)
ax.set_xticklabels(xlabels_exp_red)

ylabels_exp=sorted(df['column_count'].unique())
ylabels=np.log(ylabels_exp)
ax.set_ylabel('column_count')
ax.set_yticks(ylabels)
ax.set_yticklabels(ylabels_exp)

min_time=df[['spark_df_compute', 'pandas_df_compute']].min().min()
max_time=df[['spark_df_compute', 'pandas_df_compute']].max().max()
zlabels_log=np.linspace(np.log(min_time), np.log(max_time), 5)
zlabels=np.exp(zlabels_log)
ax.set_zlabel('compute_time (s)')
ax.set_zticks(zlabels_log)
ax.set_zticklabels(zlabels.round(2))

ax.set_title('DataFrame Mean Computation Time (s)', pad=25)
col1_patch = mpatches.Patch(color='tab:blue', label='Spark DataFrame')
col2_patch = mpatches.Patch(color='tab:red', label='Pandas DataFrame')
plt.legend(handles=[col1_patch, col2_patch])

#====second plot====
# ax=plt.fig_subplot(1, 2, 1, projection='3d')
# ax.plot_surface(X_1,
#                 X_2, 
#                 pandas_Z, 
#                 color='tab:red', 
#                 label='Pandas DataFrame')

# xlabels_exp=sorted(df['row_count'].unique())
# xlabels_exp_red=[label for idx, label in enumerate(xlabels_exp) if idx%2==0]
# xlabels=np.log(xlabels_exp_red)
# ax.set_xlabel('row_count')
# ax.set_xticks(xlabels)
# ax.set_xticklabels(xlabels_exp_red)

# ylabels_exp=sorted(df['column_count'].unique())
# ylabels=np.log(ylabels_exp)
# ax.set_ylabel('column_count')
# ax.set_yticks(ylabels)
# ax.set_yticklabels(ylabels_exp)

# min_time=df[['spark_df_compute', 'pandas_df_compute']].min().min()
# max_time=df[['spark_df_compute', 'pandas_df_compute']].max().max()
# zlabels_log=np.linspace(np.log(min_time), np.log(max_time), 5)
# zlabels=np.exp(zlabels_log)
# ax.set_zlabel('compute_time (s)')
# ax.set_zticks(zlabels_log)
# ax.set_zticklabels(zlabels.round(2))

# ax.set_title('DataFrame Count Computation Time (s)', pad=25)
# col1_patch = mpatches.Patch(color='tab:blue', label='Spark DataFrame')
# col2_patch = mpatches.Patch(color='tab:red', label='Pandas DataFrame')
# plt.legend(handles=[col1_patch, col2_patch])

# for ii in range(0,360,1):
#     if ii>=180 and ii<=270 and ii%3==0: 
#         ax.view_init(elev=10., azim=ii)
#         plt.savefig("movie/movie%d.png" % ii)
#     elif (ii<=180 or ii>=270) and ii%10==0: 
#         ax.view_init(elev=10., azim=ii)
#         plt.savefig("movie/movie%d.png" % ii)

#240 puts 0, 0 on left
ax.view_init(elev=5, azim=240)
plt.show()

Using matplotlib backend: Qt5Agg


In [7]:
df[['element_count']].corr()

,element_count
element_count,1.0
